In [12]:
# import pyspark class Row from module sql
from pyspark.sql import *


# Create some product items
item1 = Row(id=1, name="iPhone 6", price=1000.00)
item2 = Row(id=2, name="iPhone 7", price=1200.00)
item3 = Row(id=2, name="Samsung Galaxy", price=900.00)


# Create a data frame from the list Rows

df_items = spark.createDataFrame([item1,  item2, item3])

# Check automatciall inferered schema

df_items.printSchema()

# Display the dataframe with the show() function

df_items.show()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- price: double (nullable = true)

+---+--------------+------+
| id|          name| price|
+---+--------------+------+
|  1|      iPhone 6|1000.0|
|  2|      iPhone 7|1200.0|
|  2|Samsung Galaxy| 900.0|
+---+--------------+------+



In [21]:
# Create a Review row definition
Review = Row('date', 'product_id', 'user_id', 'rating', 'comment')

# Create some reviews

review1 = Review('2017-01-01', 1, 'jonh', 4, 'Very nice')
review2 = Review('2017-01-02', 1, 'karl', 3, None)
review3 = Review('2017-01-02', 1, 'adam', 5, 'Super')
review4 = Review('2017-01-03', 2, 'greg', 3, None)

# Create a data frame from the list Rows with infered schema

df_reviews_infered = spark.createDataFrame([review1,  review2, review3, review4])
df_reviews_infered.printSchema()

root
 |-- date: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- comment: string (nullable = true)



In [24]:
# Create a data frame from the list Rows with specified schema

from pyspark.sql.types import *

reviewSchema = StructType([
    StructField('date', StringType(), True),
    StructField('product_id', LongType(), False),
    StructField('user_id', StringType(), False),
    StructField('rating', IntegerType(), True),
    StructField('text', StringType(), True)
])

df_reviews = spark.createDataFrame([review1,  review2, review3, review4], schema=reviewSchema)
df_reviews.printSchema()

display(df_reviews)

root
 |-- date: string (nullable = true)
 |-- product_id: long (nullable = false)
 |-- user_id: string (nullable = false)
 |-- rating: integer (nullable = true)
 |-- text: string (nullable = true)



,date,product_id,user_id,rating,text
0,2017-01-01,1,jonh,4,Very nice
1,2017-01-02,1,karl,3,None
2,2017-01-02,1,adam,5,Super
3,2017-01-03,2,greg,3,None


### Basic operations on data frames

In [25]:
df_reviews.count()

4

In [39]:
df_filtered = df_reviews.filter(df_reviews.product_id == 1).sort(df_reviews.rating)
display(df_filtered)

,date,product_id,user_id,rating,text
0,2017-01-02,1,karl,3,None
1,2017-01-01,1,jonh,4,Very nice
2,2017-01-02,1,adam,5,Super


In [41]:
from pyspark.sql.functions import col

# Use `&` instead of `and`  and `|` instead of or

df_filtered = df_reviews.filter((col("product_id") == 1) & (col("user_id") == 'jonh'))
display(df_filtered)

,date,product_id,user_id,rating,text
0,2017-01-01,1,jonh,4,Very nice


In [52]:
from pyspark.sql.functions import asc

df_selected = df_reviews \
    .select(col("product_id"), col("rating"), col("user_id").alias("user")) \
    .sort(col("rating"))
display(df_selected)

,product_id,rating,user
0,1,3,karl
1,2,3,greg
2,1,4,jonh
3,1,5,adam


In [59]:
df_grouped = df_reviews.groupBy(col("product_id")).avg("rating")
display(df_grouped)

,product_id,avg(rating)
0,1,4.0
1,2,3.0


In [63]:
df_joined = df_reviews.join(df_items,  df_items.id == df_reviews.product_id)
display(df_joined)

,date,product_id,user_id,rating,text,id,name,price
0,2017-01-01,1,jonh,4,Very nice,1,iPhone 6,1000.0
1,2017-01-02,1,karl,3,None,1,iPhone 6,1000.0
2,2017-01-02,1,adam,5,Super,1,iPhone 6,1000.0
3,2017-01-03,2,greg,3,None,2,iPhone 7,1200.0
4,2017-01-03,2,greg,3,None,2,Samsung Galaxy,900.0


In [67]:
pandas_df = df_joined.limit(3).toPandas()
print(pandas_df)

         date  product_id user_id  rating       text  id      name   price
0  2017-01-01           1    jonh       4  Very nice   1  iPhone 6  1000.0
1  2017-01-02           1    karl       3       None   1  iPhone 6  1000.0
2  2017-01-02           1    adam       5      Super   1  iPhone 6  1000.0


In [70]:
df_joined.write.csv('/tmp/reviews.csv', mode='overwrite')

In [74]:
%%sh
cat /tmp/reviews.csv/*

2017-01-01,1,jonh,4,Very nice,1,iPhone 6,1000.0
2017-01-02,1,karl,3,,1,iPhone 6,1000.0
2017-01-02,1,adam,5,Super,1,iPhone 6,1000.0
2017-01-03,2,greg,3,,2,iPhone 7,1200.0
2017-01-03,2,greg,3,,2,Samsung Galaxy,900.0


http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame